# <font color='red'>Algoritimo de Rede Neural Arificial</font>

###### Por: Marcus Rosa Godoi

![rna.png](rna.png)

Fonte:<a href="https://medium.com/brasil-ai/entendendo-o-funcionamento-de-uma-rede-neural-artificial-4463fcf44dd0">medium.com/</a>

# 1 - Keras - Sequential Model
<a href="https://keras.io/guides/sequential_model/">keras/</a>

In [ ]:
#Import
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model


from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix
from yellowbrick.classifier import ConfusionMatrix   

## 1.1 - Criar Modelo Sequencial

Funções de ativação:

1. relu function
2. sigmoid function     # Classificação Binária
3. softmax function     # Classificação Categórica
4. tanh function
5. softplus function    #softplus(x) = log(exp(x) + 1).
6. softsign function    #softsign(x) = x / (abs(x) + 1).
7. selu function
8. elu function
9. exponential function

In [ ]:
# Criando o modelo regressivo usando KERAS

modelo = Sequential()  #uma camada apos a outra

modelo.add(Dense(units = 5, activation = 'relu', input_dim = 4))#camadas oculta1 (5 neurônios)+ camda de entrada(4)
modelo.add(Dropout(0.2))

modelo.add(Dense(units = 4,  activation = 'relu'))# camada de oculta 2 (4)
modelo.add(Dropout(0.2))

modelo.add(Dense(units = 1, activation = 'linear')) #camada de saida (1 saídas)

In [ ]:
# Ver Modelo
modelo.summary()

## 1.2 - Otimizador

#### Otimizadores disponíveis:                                 
1. SGD                              
2. RMSprop                          
3. Adam                             
4. Adadelta                       
5. Adagrad                          
6. Adamax                            
7. Nadam                             
8. Ftrl      

<a href="https://keras.io/api/optimizers/">optimezers/</a>

#### Metrics 
Regression metrics

1. MeanSquaredError class
2. RootMeanSquaredError class
3. MeanAbsoluteError class
4. MeanAbsolutePercentageError class
5. MeanSquaredLogarithmicError class
6. CosineSimilarity class
7. LogCoshError class


<a href="https://keras.io/api/metrics/">metrics/</a>

#### Loss = Erro para o treinamento.
Regression losses

1.  mean_squared_error function
2.  mean_absolute_error function
3.  mean_absolute_percentage_error function
4.  mean_squared_logarithmic_error function
5. cosine_similarity function
6. huber function
7. log_cosh function

<a href="https://keras.io/api/losses/">losses/</a>

In [ ]:
modelo.compile(optimizer = 'adam', 
                      loss = 'mean_squared_error',
                      metrics = ['mean_squared_error'])

## 1.3 - Treino

In [ ]:
# Treinando o modelo
historico = modelo.fit(previsores, 
                       preco_real, 
                       batch_size = 300, 
                       epochs = 100) #preco_real é a variavel de exemplo que queremos prever
#batch_size=64
#validation_split=0.2

## 1.4 - Imagem do Treino

In [ ]:
historico.history.keys()
plt.plot(historico.history['val_loss'])
plt.plot(historico.history['val_accuracy'])

## 1.5 Seleção de Atributos
Usando abordagem estatística univariada.

### Validação Cruzada

In [ ]:
resultados = cross_val_score(modelo, 
                         x_treino, 
                         y_treino, 
                         cv=10,
                         n_jobs= -1,
                         scoring = 'mean_squared_error')

In [ ]:
media = resultados.mean()
std = resultados.std()
print('Média: ', media)
print('Std: ', std)

In [ ]:
melhor_resultado = media

### Obter pontuação usando modelos com seleção de recursos

In [ ]:
for i in range(1, x_treino_poly.shape[1]+1, 1):
    # Select i atributos
    select = SelectKBest(score_func=chi2, k=i) #chi2 = qui-quadrado
    select.fit(x_treino_poly, y_treino)
    x_treino_poly_selected = select.transform(x_treino_poly)

    # Modelo com i atributos selecionados
    modelo.fit(x_treino_poly_selected, y_treino)
    resultados = cross_val_score(modelo, x_treino_poly_selected, y_treino, cv=10)
    media = resultados.mean()
    std = resultados.std()
    print('Média: ', media)
    print('Std: ', desvio)
    print('CV acuracia: ', media, '+/- ', std)
    
    # Salvar resultados se melhorar a pontuação
    if np.mean(resultados) > melhor_resultado:
        melhor_resultado = media
        melhor_std = std
        k_atributos_melhor_resultado = i
    elif media == melhor_resultado:
        if std <  melhor_std:
            melhor_resultado = media
            melhor_std = std
            k_atributos_melhor_resultado = i

In [ ]:
print('Número de atributos com melhor pontuação: ', k_atributos_melhor_resultado)

### Selecionar Atributos

In [ ]:
select = SelectKBest(score_func = chi2, k = k_atributos_melhor_resultado)
select.fit(x_treino_poly, y_treino)
x_treino_poly_selected = select.transform(x_treino_poly)

### Treinar Modelo

In [ ]:
modelo = Sequential()  #uma camada apos a outra
modelo.add(Dense(units = 5, activation = 'relu', input_dim = 4))#camadas oculta1 (5 neurônios)+ camda de entrada(4)
modelo.add(Dropout(0.2))
modelo.add(Dense(units = 4,  activation = 'relu'))# camada de oculta 2 (4)
modelo.add(Dropout(0.2))
modelo.add(Dense(units = 1, activation = 'linear')) #camada de saida (1 saídas)
modelo.compile(optimizer = 'adam', 
                      loss = 'mean_squared_error',
                      metrics = ['mean_squared_error'])


modelo.fit(x_treino_poly_selected, y_treino,batch_size = 512,epochs = 2000))

### Resultados


In [ ]:
resultados = cross_val_score(modelo, 
                         x_treino_poly_selected, 
                         y_treino, 
                         cv=10,
                         n_jobs= -1,
                         scoring = 'mean_squared_error')

### Curva de Aprendizado

In [ ]:
title = "Curva de Aprendizado (RNA)"
cv = 10
plot_learning_curve(modelo, 
                    title, 
                    x_treino_poly_selected,
                    y_treino, 
                    ylim = (0.7, 1.01), 
                    cv = cv, 
                    n_jobs = -1);

### Curva de Validação

In [ ]:
title = 'Curva de Validação (RNA)'
param_name = 'optimizer'
param_range = ['SGD', 'RMSprop', 'adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftr1'] 
cv = 10
plot_validation_curve(estimator=modelo, 
                      title = title, 
                      X = x_treino_poly_selected, 
                      y = y_treino, 
                      param_name = param_name, 
                      ylim = (0.5, 1.01), 
                      param_range = param_range);

### Fazer a mesma transformação com os dados de teste.

## 1.6 - Verificando a exatidão no modelo nos dados de teste

In [ ]:
previsao1 = modelo.predict(x_teste)
previsao2 = resultado.predict(x_teste)

In [ ]:
print(preco_real.mean()) #preco_real -> exemplo de saída
print('Média: ', previsao.mean())
print('Std: ', previsao.std())

In [ ]:
print(preco_real.mean()) #preco_real -> exemplo de saída
print('Média: ', resultado.mean())
print('Std: ', resultado.std())

In [ ]:
print('MAE:', mean_absolute_error(y_teste, previsao).round(8)  )
print('MSE:', mean_squared_error(y_tteste, previsao).round(8) )  
print('RMSE:', np.sqrt(mean_squared_error(y_teste, previsao)).round(8))

In [ ]:
resultado = modelo.evaluate(x_teste, y_teste)#usando ferramentas do keras
resultado

## 1.7 - Tuning

# Fim

### Obrigado - Marcus Rosa Godoi 